In [5]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import datetime
fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'
storage = "/Volumes/easystore/Drones"
coi=['Incident No.','drone-incident-id','Description','drone-type','cfs-date', 'drone-date','Block Location','drone-address','Full FIPS (block)','drone-FIPS','drone-id','Latitude','Longitude']
orphan_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../../data/outputs/outputs-flights-without-matching-cfs_matched-by-id.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
calls_for_service = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/calls-for-service-with-census-data.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)

flight_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../../data/outputs/outputs_drone-seconds-block-count-per-flight.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)
cv_blocks = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"../../../data/mapping/cv-blocks.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

Loading data: 0it [00:00, ?it/s]

In [6]:
orphan_flight_blocks = flight_blocks[flight_blocks['id'].isin(orphan_flights['id'])]

In [7]:
calls_for_service["cfs-date"] = pd.to_datetime(calls_for_service["Date"])
orphan_flights["drone-date"] =  pd.to_datetime(orphan_flights['date'] + " " + orphan_flights['time_s'])

In [8]:
pc = []
for _, row in tqdm(orphan_flights.sample(frac=1).iterrows()):
    ceiling = row['drone-date'] + datetime.timedelta(minutes = 1)
    floor = row['drone-date'] - datetime.timedelta(minutes = 30)


    mask = (calls_for_service['cfs-date'] > floor) & (calls_for_service['cfs-date'] < ceiling)
    possible_calls = calls_for_service.loc[mask].copy()
    possible_calls['drone-id'] = row['id']
    possible_calls['drone-incident-id'] = row['incident_id']
    possible_calls['drone-date'] = row['drone-date']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-address'] = row['address_map']
    possible_calls['drone-FIPS'] = row['Full FIPS (block)']

    # possible_calls = possible_calls[possible_calls['Full FIPS (block)']==row['Full FIPS (block)']].copy()
    
    pc.append(possible_calls)
    

0it [00:00, ?it/s]

In [9]:
possible_calls = pd.concat(pc)

In [10]:
flew_near_cfs = []
for _, flight in tqdm(orphan_flights.iterrows()):
    possible_cfs_for_flight = possible_calls[possible_calls['drone-id']==flight['id']]
    orphan_flight = orphan_flight_blocks[orphan_flight_blocks['id']==flight['id']]
    possible_cfs_for_flight_flew_close = possible_cfs_for_flight[possible_cfs_for_flight['Full FIPS (block)'].isin(orphan_flight['GEOID20'])]
    flew_near_cfs.append(possible_cfs_for_flight_flew_close)

    

0it [00:00, ?it/s]

In [11]:
possible_calls_by_proximity = pd.concat(flew_near_cfs)
possible_calls_by_proximity[coi]

,Incident No.,drone-incident-id,Description,drone-type,cfs-date,drone-date,Block Location,drone-address,Full FIPS (block),drone-FIPS,drone-id,Latitude,Longitude
124874,210730L00063426,CVL63427,[Protected Call Type],Domestic Violence,2021-07-30 16:59:59.423,2021-07-30 17:04:00,600 BLOCK I ST,600 I St.,060730127001003,060730127001003,3f07b495a2ef58c78a6e091996d4a8aa,32.627663,-117.088104
152512,210730L00063424,CVL63427,Found,Domestic Violence,2021-07-30 16:50:52.003,2021-07-30 17:04:00,300 BLOCK BRIGHTWOOD AV,600 I St.,060730127003000,060730127001003,3f07b495a2ef58c78a6e091996d4a8aa,32.638057,-117.085887
145269,210730L00063417,NaN,Collision - No Details,NaN,2021-07-30 16:27:44.990,2021-07-30 16:33:00,OTAY LAKES RD & BONITA RD,"Racquetball courts, 900 Otay Lakes Rd",060730134111013,060730134151000,4daa5e5a1bf0bb212fb633f367ed09a0,32.66227,-117.029029
120777,210729L00062953,2109480,Discharge of Firearm,Negligent Discharge of Firearm,2021-07-29 06:37:17.437,2021-07-29 07:01:00,900 BLOCK FIFTH AV,900 Fifth Ave,060730131044000,060730131044000,22e84c726a9ee87e431e1b7fd05cb876,32.618154,-117.07996
94507,210728L00062809,L062810,Traffic Stop,BRUSH FIRE,2021-07-28 17:13:50.303,2021-07-28 17:17:00,300 BLOCK NAPLES ST,4TH/MAIN,060730131023003,060730208133010,d4f6c7662dd18682f8189a0a5b4ca4d7,32.613534,-117.069115
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4201,230822L00071689,CVL070682,Disturbance - Homeless-Related,Person causing a disturbance,2023-08-22 11:42:53.827,2023-08-22 12:05:00,300 BLOCK FOURTH AV,706 F Street,060730127003000,060730125011000,8a3ef045c6d06c7d1ceeb69c32c34094,32.639993,-117.083982
57404,230818L00070716,CV2311196,Resist/Delay/Obstruct Officer,Unknown Problem,2023-08-18 19:41:43.430,2023-08-18 19:49:00,1 BLOCK NAPLES ST,7 Naples St.,060730131021000,060730131024004,43a7c1e800e3e6a1475dd3b08d60d7e9,32.6161,-117.060965
83658,230817L00070310,CVL2311139,Indecent Exposure,"415 Person, Veh Pursuit",2023-08-17 13:38:43.150,2023-08-17 13:46:00,1600 BLOCK BROADWAY,540 Auto Park Dr,060730132051013,060730133173051,0b23301d6f257c73b109798c5214a1d7,32.596398,-117.076633
101239,230817L00070314,CVL2311139,Non-Injury Collision,"415 Person, Veh Pursuit",2023-08-17 13:45:30.410,2023-08-17 13:46:00,FIFTH AV & OXFORD ST,540 Auto Park Dr,060730132062000,060730133173051,0b23301d6f257c73b109798c5214a1d7,32.607965,-117.07591


In [13]:
matched = possible_calls_by_proximity[coi]['drone-id']
orphan_flights_not_close = orphan_flights[~orphan_flights['id'].isin(matched)]



In [ ]:
pc = []
for _, row in tqdm(orphan_flights_not_close.sample(frac=1).iterrows()):
    ceiling = row['drone-date'] + datetime.timedelta(minutes = 1)
    floor = row['drone-date'] - datetime.timedelta(minutes = 30)

    mask = (calls_for_service['cfs-date'] > floor) & (calls_for_service['cfs-date'] < ceiling)
    possible_calls = calls_for_service.loc[mask].copy()
    possible_calls['drone-id'] = row['id']
    possible_calls['drone-incident-id'] = row['incident_id']
    possible_calls['drone-date'] = row['drone-date']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-type'] = row['type']
    possible_calls['drone-address'] = row['address_map']
    possible_calls['drone-FIPS'] = row['Full FIPS (block)']
    
    # possible_calls = possible_calls[possible_calls['Full FIPS (block)']==row['Full FIPS (block)']].copy()
    
    pc.append(possible_calls)
    

In [ ]:
possible_calls = pd.concat(pc)

In [ ]:
possible_calls[possible_calls['drone-id']=='c7ea1840aae576682aeb4b31e6fe5153'][coi]

In [ ]:
compiled_flights =[] 

for r_id in possible_calls['drone-id'].sample(50).drop_duplicates().values:
    
    d = gpd.read_file(f"/Volumes/easystore/Drones/flights/kml/{r_id}.kml",driver='KML')
    d['id'] = r_id
    compiled_flights.append(d)


In [ ]:
paths = pd.concat(compiled_flights,ignore_index=True)
paths = paths[paths['Name']!="Home"]
paths[['geometry','id']].drop_duplicates().to_csv('./no-cfs-flights.csv',index=False)

In [ ]:
possible_calls[possible_calls['drone-id'].isin(paths['id'])][coi].to_csv('./possible-calls.csv',index=False)